In [ ]:
%matplotlib inline
import chainer as C
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pickle, gzip
with gzip.open('/data/mnist.pkl.gz') as f:
    unpick = pickle._Unpickler(f)
    unpick.encoding = 'latin1'
    train, valid, test = unpick.load()

In [ ]:
train[0].shape

In [ ]:
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(train[0][i].reshape((28, 28)))
    plt.axis('off')

In [ ]:
class Network(C.Chain):
    def __init__(self):
        super().__init__(
            a=C.links.Linear(784, 128),
            b=C.links.Linear(128, 128),
            c=C.links.Linear(128, 128),
            final=C.links.Linear(128, 10),
        )

    def __call__(self, x):
        w = x
        w = C.functions.tanh(self.a(w))
        w = C.functions.tanh(self.b(w))
        w = C.functions.tanh(self.c(w))
        return self.final(w)

In [ ]:
batch_size = 32

network = Network()
opt = C.optimizers.Adam()
opt.use_cleargrads()
opt.setup(network)

accuracies = []
for i in range(0, len(train[0]), batch_size):
    batch_x = C.Variable(train[0][i:(i + batch_size)])
    batch_y = C.Variable(train[1][i:(i + batch_size)].astype(np.int32))

    network.cleargrads()
    z = network(batch_x)
    c = C.functions.softmax_cross_entropy(z, batch_y)
    c.backward()
    opt.update()

    a = 100 * C.functions.accuracy(z, batch_y)
    #print("Loss = %.2f, Accuracy = %.2f%%" % (c.data, a.data))
    accuracies.append(float(a.data))

valid_accuracy = 100 * C.functions.accuracy(
    network(C.Variable(valid[0])), C.Variable(valid[1].astype(np.int32)))
print("Validation Accuracy = %.2f%%" % (a.data))

In [ ]:
plt.plot(accuracies)